In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.metrics import  f1_score

Using TensorFlow backend.


In [2]:
data = pd.read_csv("ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)
data.dropna(inplace=True)
df=pd.DataFrame({'word':data.word,'tag':data.tag,'pos':data.pos})
df.reset_index(drop=True,inplace=True)

'''
plt.figure(figsize=(10,5))
ax = sns.countplot('tag', data=df.loc[df['tag'] != 'O'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
plt.tight_layout()
'''
#plt.show()


b'Skipping line 281837: expected 25 fields, saw 34\n'


'\nplt.figure(figsize=(10,5))\nax = sns.countplot(\'tag\', data=df.loc[df[\'tag\'] != \'O\'])\nax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")\nplt.tight_layout()\n'

In [3]:
converted_data = []

i = 0
cnt = 0
while i < len(df):
    ls = []
    while (i + 1 < len(df)) & (df.word[i] != '.'):
        ls.append(df.word[i]) 
        i += 1
    i += 1
    converted_data.append(ls)
    


In [10]:
max_len=0
for sent in converted_data[:1000]:
    if len(sent)>max_len:
        max_len=len(sent)
max_len

60

In [5]:
class callback(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0
        self.prev=0
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch%10==9:
            print('Loss after epoch {}: {}'.format(self.epoch+1, loss-self.prev))
        self.prev=loss
        self.epoch += 1

In [130]:
size = 5
model_w = Word2Vec(converted_data,
                 size=size,
                 window=5,
                 min_count=1,
                 sample=0.001,
                 seed=1,
                 workers=4,
                 sg=0,
                 negative=5,
                 iter=100,
                 compute_loss=True,
                 callbacks=[callback()])

Loss after epoch 10: 394708.5
Loss after epoch 20: 348754.5
Loss after epoch 30: 336043.0
Loss after epoch 40: 321351.0
Loss after epoch 50: 285576.0
Loss after epoch 60: 294918.0
Loss after epoch 70: 294024.0
Loss after epoch 80: 279892.0
Loss after epoch 90: 278656.0
Loss after epoch 100: 277294.0


In [131]:
t=0
corpus = []
for sent in converted_data:
    temp=[]
    for word in sent:
        t+=1
        #scaling = MinMaxScaler(feature_range=(-1, 1)).fit(model[word].reshape(-1, 1))
        wrd = model_w[word]
        temp.append(wrd)
    corpus.append(np.array(temp))
corpus=np.array(corpus)

In [132]:
corpus_new=np.squeeze(tf.keras.preprocessing.sequence.pad_sequences(corpus, maxlen=96, padding='post')).astype(np.float32)



In [133]:
corpus_new.shape

(47896, 96, 5)

In [13]:
unq_labels=list(set(df.tag))
unq_labels.remove('O')

In [14]:
dct={}
for idx,x in enumerate(unq_labels):
    dct[x]=idx+1

In [126]:
i=0
prep_label=[]
while i < len(df):
    ls = []
    while (i + 1 < len(df)) & (df.word[i] != '.'):
        if df.tag[i] in unq_labels:
            
            ls.append(dct[df.tag[i]]) 
        else:
            ls.append(0)
        i += 1
    i += 1
    prep_label.append(ls)

In [127]:
prep_label_new=np.squeeze(tf.keras.preprocessing.sequence.pad_sequences(prep_label, maxlen=96, padding='post',value=0))
prep_label_new.shape

(47896, 96)

In [138]:
BATCH_SIZE=128
rnn_units=1024

In [139]:
def build_model( rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(rnn_units,
                    return_sequences=True,
                    stateful=False,
                    recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(96)
        ])
    return model
model = build_model(
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)


In [140]:

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
#model.summary()

In [142]:
EPOCHS=1
model.fit(corpus_new,prep_label_new, epochs=EPOCHS)

Train on 47896 samples
47896/47896 [==============================] - 100s 2ms/sample - loss: 0.0978 - sparse_categorical_accuracy: 0.9737


In [145]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues = np.array([])

# iterate through test set, make predictions based on trained model


pred = model.predict(corpus_new[:20000])
pred_max = tf.argmax(tf.nn.softmax(pred), 2).numpy().flatten()
y_true = prep_label_new.flatten()

preds = np.concatenate([preds, pred_max])
y_trues = np.concatenate([y_trues, y_true])

# remove padding from evaluation
remove_padding = [(p, y) for p, y in zip(preds, y_trues) if y != 0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p, r_t))
print(classification_report(r_p, r_t))

ResourceExhaustedError: OOM when allocating tensor with shape[20000,96,96] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Softmax]